In [1]:
!pip install transformers seqeval[gpu] pandas numpy sklearn mlflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid

# Import

In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, PreTrainedTokenizer, PreTrainedModel
import mlflow
from seqeval.metrics import precision_score, recall_score

In [50]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Load the dataset

In [223]:
data = pd.read_csv("./data/labeled_doctor_req_chatgpt.csv")
data.head()

,sentence,label
0,یک دکتر مرد خوب برای ارتوپد در کرمانشاه سراغ د...,"O,O,B-gen,O,O,O,O,B-cit,O,O,O"
1,آیا می توانید یک دکتر برای آسم معرفی کنید ؟,"O,O,O,O,O,O,O,O,O,O"
2,کسی یک دکتر مرد برای افسردگی در خرم آباد یا مش...,"O,O,O,B-gen,O,O,O,B-cit,I-cit,O,B-cit,O,O,O"
3,آیا می توانید یک دکتر برای روانپزشکی معرفی کنید ؟,"O,O,O,O,O,O,O,O,O,O"
4,آیا می توانید یک دکتر برای زنان معرفی کنید ؟,"O,O,O,O,O,O,B-gen,O,O,O"


In [52]:
label2id = {
    'O': 0,
    'B-gen': 1,
    'I-gen': 2,
    'B-cit': 3,
    'I-cit': 4
}

In [53]:
id2label = {
    0: 'O',
    1: 'B-gen',
    2: 'I-gen',
    3: 'B-cit',
    4: 'I-cit'
}

In [272]:
data = pd.read_csv("./data/form_data.csv", index_col=0)
data.head()

,sentence,label
0,سلام یک دکتر فوق تخصص زنان خانوم تو اصفهان سمت...,"O,O,O,B-srt,I-srt,B-spy,B-gnd,O,B-cty,O,B-nhd,..."
1,سلام . یک متخصص مو داخل قم سمت پارک ملی برام پ...,"O,O,O,B-srt,B-spy,O,B-cty,O,B-nhd,I-nhd,O,O,O,..."
2,سلام . یک تخصص پوست و مو مرد درون قم سمت پارک ...,"O,O,O,B-srt,B-spy,O,B-spy,B-gnd,O,B-cty,O,B-nh..."
3,سلام ، یک متخصص تغذیه شهر علویجه معرفی کن با ت...,"O,O,O,B-srt,B-spy,O,B-cty,O,O,O,B-inc,I-inc,O,..."
4,سلام ، یک متخصص تغذیه مرد شهر گراش معرفی کن بر...,"O,O,O,B-srt,B-spy,B-gnd,O,B-cty,O,O,O,O,B-cnd,..."


In [222]:
# Extract all unique BIO tags used in the dataset
unique_bio_tags = set(tag for bio_tag in data['label'] for tag in bio_tag.split(','))

# Convert to a sorted list
unique_bio_tags_list = sorted(unique_bio_tags)

unique_bio_tags_list

['B-apt',
 'B-cnd',
 'B-cty',
 'B-gnd',
 'B-inc',
 'B-nhd',
 'B-spy',
 'B-srt',
 'B-trt',
 'B-vtp',
 'B-wtt',
 'I-apt',
 'I-cnd',
 'I-inc',
 'I-nhd',
 'I-spy',
 'I-srt',
 'I-trt',
 'I-vtp',
 'I-wtt',
 'O']

In [56]:
label2id = {
    'O': 0,
    'B-apt': 1,
    'I-apt': 2,
    'B-cty': 3,
    'I-cty': 4,
    'B-cnd': 5,
    'I-cnd': 6,
    'B-gnd': 7,
    'I-gnd': 8,
    'B-inc': 9,
    'I-inc': 10,
    'B-nhd': 11,
    'I-nhd': 12,
    'B-srt': 13,
    'I-srt': 14,
    'B-spy': 15,
    'I-spy': 16,
    'B-trt': 17,
    'I-trt': 18,
    'B-vtp': 19,
    'I-vtp': 20,
    'B-wtt': 21,
    'I-wtt': 22
}

id2label = {v: k for k, v in label2id.items()}

In [57]:
data.columns = ['sentence', 'label']

In [58]:
data['label'] = data['label'].apply(lambda x: x.replace(" ", ","))

# Preprocess

In [59]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

/home/user/anaconda3/envs/nlu/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Define functions

In [60]:
def tokenize_and_preserve_labels(sentence: str, text_labels: str, tokenizer: PreTrainedTokenizer) -> tuple[list[str], list[str]]:
    tokenized_sentence = []
    labels = []
    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [61]:
class dataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer: PreTrainedTokenizer, max_len: int) -> None:
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index :int):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.label[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

## Split into Train, Test

In [62]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (93, 2)
TRAIN Dataset: (74, 2)
TEST Dataset: (19, 2)


In [63]:
training_set[0]

{'ids': tensor([    2,  4285,  1012,  4642,  4283,  9084,  4124,  1379,  2799,  4246,
          4124,  1379,  4197,  7811, 17953,  2800, 38905,  3404,  9747,  8444,
          2840,  3803, 14114,  1012,  6599, 12139,  1379,  3757, 16172,  4435,
         12139,  1012,  5072,  5505,  2820, 41774,  2008,  1379,  8512,  8113,
          5442,  2820,  3151, 12139,  1012,  4202,  2820,  5671, 14114,  1012,
          3130,  2791,  2829,  3551, 23537, 75915,  1012,     4,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [64]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
سلام        O
.           O
یه          O
دکتر        O
تخصص        B-srt
خوب         B-trt
و           O
با          B-trt
تجربه       I-trt
خوب         O
و           O
خوش         B-trt
اخلاق       I-trt
میخوام      O
که          O
بتونه       O
مشکل        O
ناراحتی     B-cnd
معده        I-cnd
رو          O
حل          O
کنه         O
.           O
شیراز       B-cty
باشه        O
و           O
سمت         O
باهنر       B-nhd
جنوبی       I-nhd


In [65]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Define and track models with mlflow

In [224]:
EPOCHS = 25
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
FREAZING_LAYERS = 6

In [225]:
mlflow.set_tracking_uri("mlflow")
mlflow.set_experiment("NER")

Traceback (most recent call last):
  File "/home/user/anaconda3/envs/nlu/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 317, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/envs/nlu/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 410, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/envs/nlu/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 1341, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/envs/nlu/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 1334, in _read_helper
    result = read_yaml(root, file_na

<Experiment: artifact_location='/home/user/Desktop/hammasir-project/mlflow/968568366775758109', creation_time=1724758511411, experiment_id='968568366775758109', last_update_time=1724758511411, lifecycle_stage='active', name='NER', tags={}>

## Writing Functions

In [226]:
def initialize_model(id2label: dict, label2id: dict, training_set: dataset, freeze_layers: int = 0) -> tuple[PreTrainedModel, torch.Tensor]:
    model = BertForTokenClassification.from_pretrained(
        'HooshvareLab/bert-fa-base-uncased', 
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id
    )
    
    # Freeze the first `freeze_layers` transformer layers
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False

    for i in range(freeze_layers):
        for param in model.bert.encoder.layer[i].parameters():
            param.requires_grad = False

    model.to(device)
    ids = training_set[0]["ids"].unsqueeze(0)
    mask = training_set[0]["mask"].unsqueeze(0)
    targets = training_set[0]["targets"].unsqueeze(0)
    ids = ids.to(device)
    mask = mask.to(device)
    targets = targets.to(device)
    outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
    initial_loss = outputs[0]
    return model, initial_loss

In [227]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(optimizer: torch.optim.Adam, max_norm: int, training_loader: DataLoader, model: PreTrainedModel) -> tuple[PreTrainedModel, float, float]:
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_norm
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    return model, epoch_loss, tr_accuracy

In [228]:
def valid(model: PreTrainedModel, testing_loader: DataLoader, device: str, id2label: dict, label2id: dict) -> tuple[list[str], list[str], float, float]:
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps

    return labels, predictions, eval_loss, eval_accuracy

In [229]:
def predict(sentence: str, model: PreTrainedModel, tokenizer: BertTokenizer, id2label: dict, device: str) -> tuple[str, list[str]]:
  inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  word_level_predictions = []
  for pair in wp_preds:
    if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
      # skip prediction
      continue
    else:
      word_level_predictions.append(pair[1])

  # we join tokens, if they are not special ones
  str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
  return str_rep, word_level_predictions

# Base Model

In [230]:
mlflow.transformers.autolog(disable=True)
with mlflow.start_run():
    mlflow.log_params({
        'EPOCHS': EPOCHS,
        'LEARNING_RATE': LEARNING_RATE,
        'MAX_GRAD_NORM': MAX_GRAD_NORM,
        'FREAZING_LAYERS': FREAZING_LAYERS
    })
    model, initial_loss = initialize_model(label2id=label2id, id2label=id2label, training_set=training_set, freeze_layers=FREAZING_LAYERS)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
    
    for i in range(EPOCHS):
        model, epoch_loss, tr_accuracy = train(optimizer, MAX_GRAD_NORM, training_loader, model)
        mlflow.log_metric(f'loss_epoch{i+1}', epoch_loss)
        mlflow.log_metric(f'accuracy_epoch{i+1}', tr_accuracy)
        print("tr_accuracy = ", tr_accuracy)    
        
    labels, predictions, eval_loss, eval_accuracy = valid(model, testing_loader, device, id2label, label2id)
    mlflow.log_metric('eval_loss', eval_loss)
    mlflow.log_metric('eval_accuracy', eval_accuracy)
    precision = precision_score([labels], [predictions], average=None)
    recall = recall_score([labels], [predictions], average=None)
    mlflow.log_metric('cit_precision', precision[0])
    mlflow.log_metric('cit_recall', recall[0])
    mlflow.log_metric('gen_precision', precision[1])
    mlflow.log_metric('gen_recall', recall[1])
    mlflow.pytorch.log_model(model, 'model')
    print("eval_accuracy = ", eval_accuracy)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training loss per 100 training steps: 3.233670473098755
tr_accuracy =  0.5419695997049545
Training loss per 100 training steps: 0.5881020426750183
tr_accuracy =  0.6372003120162022
Training loss per 100 training steps: 0.4729653000831604
tr_accuracy =  0.6412305876920411
Training loss per 100 training steps: 0.36786866188049316
tr_accuracy =  0.6469864243189588
Training loss per 100 training steps: 0.433521032333374
tr_accuracy =  0.6468173681283788
Training loss per 100 training steps: 0.29667139053344727
tr_accuracy =  0.6721701555047517
Training loss per 100 training steps: 0.22286473214626312
tr_accuracy =  0.7125645737720623
Training loss per 100 training steps: 0.16869600117206573
tr_accuracy =  0.7478057272825701
Training loss per 100 training steps: 0.16590897738933563
tr_accuracy =  0.7900252634324513
Training loss per 100 training steps: 0.1696079671382904
tr_accuracy =  0.8121903545889342
Training loss per 100 training steps: 0.14075171947479248
tr_accuracy =  0.843747658342

2024/08/31 17:16:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpmbq4qri9/model/data, flavor: pytorch). Fall back to return ['torch==2.4.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/08/31 17:16:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


eval_accuracy =  0.9259790317038761


In [271]:
sentence = "سلام یک دکتر دکترای حرفه‌ای تغذیه خانم تو تهران سمت معالی آباد میخوام که تجربه کافی داشته باشه و برای پس فردا بهم نوبت بده و با نیرو های مسلح قرارداد داشته باشه و تایم انتظار توی مطب کمتر از نیم ساعت باشه."
sentence_out, word_level_predictions = predict(sentence, model, tokenizer, id2label, device)
print(sentence_out, word_level_predictions, sep="\n")

سلام یک دکتر دکترای حرفهای تغذیه خانم تو تهران سمت معالی اباد میخوام که تجربه کافی داشته باشه و برای پس فردا بهم نوبت بده و با نیروهای مسلح قرارداد داشته باشه و تایم انتظار توی مطب کمتر از نیم ساعت باشه .
['O', 'O', 'O', 'B-srt', 'B-spy', 'B-spy', 'B-gnd', 'O', 'B-cty', 'O', 'B-nhd', 'B-nhd', 'I-nhd', 'O', 'O', 'B-trt', 'I-trt', 'O', 'O', 'O', 'O', 'B-apt', 'B-apt', 'O', 'O', 'O', 'O', 'O', 'B-inc', 'I-inc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-wtt', 'I-wtt', 'I-wtt', 'I-wtt', 'O', 'O']
